# Нейросеть для сегментации изображений

## Описание проекта

Задача проекта сводится к классификации каждого пикселя. Необходимо определить к какому объекту (классу) он принадлежит. После, выделить на изображении части принадлежащие одному объекту, т.е. сегментировать изображение.
Задача обучения сводится к минимизации функции ошибки на этапе классификации пикселей.

### Входные данные

•	Датасет в папке `fishki_labelme`:
- набор из 140 изображений (2448x2448x3 JPG);
- файлы разметки в формате `.json` из [labelme](https://github.com/wkentaro/labelme);
- файл `obj.names` с именами объектов/классов:
    - __background__ - фоновые пиксели;
    - fishka - пиксели области фишки;
    - defect - пиксели области дефекта.

•	Скрипт `01_generate_dataset.py` - генерирует датасет для обучения НС в формате [Pascal VOC](http://host.robots.ox.ac.uk/pascal/VOC/). Он также разбивает выборку на train (95%) и val и test (5%). Последние 2 выборки одинаковые (val=test).

### Задачи

**Задача №1:** _Подготовка датасета_

Скрипт `01_generate_dataset.py` нужно модифицировать (или написать свой) чтобы он разбивал исходный набор отдельно на train(80%) val(10%) и test(10%).
Выходными данными должны являться:
Готовый к обучению датасет в формате Pascal VOC; 

**Задача №2:** _Обучение НС_

Необходимо обучить НС сегментатора на датасете из задачи №1
Фреймворк машинного обучения и библиотеки можно использовать любые по желанию. Необходимо обосновать выбор.
Входное разрешение нейросети при обучении необходимо также выбрать и обосновать.

Выходными данными должны являться 
-	обученная нейросеть сегментации с train датасетом из задачи №1;
-	лог обучения (графики функции потерь и mIoU от эпохи);
-	расчет метрик по сегментации на val датасете (IoU по каждому классу отдельно и mIoU);
-	расчет метрик по сегментации на test датасете (IoU по каждому классу отдельно и mIoU);

**Задача №3:** _Инференс НС_

Необходимо прогнать изображения из тестового датасета через обученную в задаче №2 нейросеть сегментатора и получить визуализации.
При выполнении задания можно использовать средства фреймворка машинного обучения (`PyTorch`, `Tensorflow`), либо сконвертировать обученную НС в формат ONNX.

Выходными данными должны являться изображения из полученного в задаче №1 test датасета размеченные обученной в задаче №2 нейросетью.

По результату тестового задания должен быть представлен краткий отчет с описанием выполненных работ, результатов тестирования НС и примерами изображений размеченных нейросетью.

Примечание: в данном случае для примера сделана просто визуализация разметки, вы должны будете сделать раскраску по результатам сегментации входных изображений нейросетью.
___

## Подключаем необходимые модули

In [ ]:
# импорт основных библиотек
import os
import glob
import numpy as np
import matplotlib.pyplot as plt

# импорт спец. библиотек и функций
import tensorflow as tf
from skimage import measure
from skimage.io import imread, imsave, imshow
from skimage.transform import resize
from skimage.filters import gaussian
from skimage.morphology import dilation, disk
from skimage.draw import polygon, polygon_perimeter

# проверка наличия GPU-ускорителя
print(f'GPU is {"ON" if tf.config.list_physical_devices("GPU") else "OFF" }')

## Подготовим набор данных для обучения

In [ ]:
CLASSES = 3 # кол-во классов + один класс обозначающий задний план
COLORS = ['black', 'red', 'green'] # цветовое обозначение классов

SAMPLE_SIZE = (256, 256) # размер входного изображения для НС
OUTPUT_SIZE = (2448, 2448) # размер изображения на выходе НС

In [ ]:
# функция загрузки и преобразования фото и маски
def load_images(image, mask):
    image = tf.io.read_file(image) # чтение фото
    image = tf.io.decode_jpeg(image)
    image = tf.image.resize(image, OUTPUT_SIZE)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = image / 255.0 # нормализация фото (отмасштабировали будущие признаки)

    # аналогичная операция выполняется для маски
    mask = tf.io.read_file(mask)
    mask = tf.io.decode_png(mask)
    mask = tf.image.rgb_to_grayscale(mask)
    mask = tf.image.resize(mask, OUTPUT_SIZE)
    mask = tf.image.convert_image_dtype(mask, tf.float32)

    masks = []
    uniq_values, uniq_id = tf.unique(tf.reshape(mask, [-1]))

    for tone in [ 0., 38., 75.]:
        masks.append(tf.where(tf.equal(mask, tone), 1.0, 0.0))
    
    masks = tf.stack(masks, axis=2)
    masks = tf.reshape(masks, OUTPUT_SIZE + (CLASSES,))

    return image, masks

In [ ]:
mask = 'fishki_voc_dataset/SegmentationClass/00000001.png'
mask = tf.io.read_file(mask)
mask = tf.io.decode_png(mask)
mask = tf.image.rgb_to_grayscale(mask)
mask = tf.image.resize(mask, OUTPUT_SIZE)
mask = tf.image.convert_image_dtype(mask, tf.float32)

masks = []
uniq_values, uniq_id = tf.unique(tf.reshape(mask, [-1]))
print(uniq_values)

for tone in [ 0., 38., 75.]:
    masks.append(tf.where(tf.equal(mask, tone), 1.0, 0.0))
    print(tf.where(tf.equal(mask, tone), 1.0, 0.0))    

masks = tf.stack(masks, axis=2)
masks = tf.reshape(masks, OUTPUT_SIZE + (CLASSES,))


In [ ]:
#plt.imshow(mask)
fig, ax = plt.subplots(nrows = 1, ncols = 3, figsize=(15, 5), dpi=125)
ax[0].imshow(masks[:, :, 0])
ax[0].set_axis_off()
ax[1].imshow(masks[:, :, 1])
ax[1].set_axis_off()
ax[2].imshow(masks[:, :, 2])
ax[2].set_axis_off()
plt.show()

In [ ]:
# функция аугментация фото и маски, соответственно
def augmentate_images(image, masks):
    # увеличение масштаба на случайную величину
    random_crop = tf.random.uniform((), 0.8, 1)
    image = tf.image.central_crop(image, random_crop)
    masks = tf.image.central_crop(masks, random_crop)

    # отражение по горизонтали
    random_flip = tf.random.uniform((), 0, 1)
    if random_flip >= 0.5:
        image = tf.image.flip_left_right(image)
        masks = tf.image.flip_left_right(masks)

    # назначение входного размера фото и маски
    image = tf.image.resize(image, SAMPLE_SIZE)
    masks = tf.image.resize(masks, SAMPLE_SIZE)

    return image, masks

In [ ]:
# загрузка имён фото и соответствующих масок
def get_image_dataset(sample):
    images = []
    masks  = []
    
    # загрузка имён фото и соответствующих масок
    file = open('fishki_voc_dataset/ImageSets/Segmentation/' + sample + '.txt', 'r')
    for line in file:
        images.append('fishki_voc_dataset/JPEGImages\\' + line[:-1] + '.jpg')
        masks.append('fishki_voc_dataset/SegmentationClass\\'+ line[:-1] +'.png')
    file.close()

    # формирование набора данных
    images_dataset = tf.data.Dataset.from_tensor_slices(images)
    masks_dataset  = tf.data.Dataset.from_tensor_slices(masks)
    dataset = tf.data.Dataset.zip((images_dataset, masks_dataset))

    return dataset

In [ ]:
# загрузка датасетов
train = get_image_dataset('train')
train = train.map(load_images, num_parallel_calls=tf.data.AUTOTUNE)       # загрузка данных в память с помощью функции load_images
#train = train.repeat(60)                                                  # копирование датасета в памяти N раз
train = train.map(augmentate_images, num_parallel_calls=tf.data.AUTOTUNE) # аугментация датасета с помощью функции augmentate_images
train = train.batch(16)

valid = get_image_dataset('val')
valid = valid.map(load_images, num_parallel_calls=tf.data.AUTOTUNE)
valid = valid.map(augmentate_images, num_parallel_calls=tf.data.AUTOTUNE) # аугментация датасета с помощью функции augmentate_images
valid = valid.batch(16)

test = get_image_dataset('test')
test = test.map(load_images, num_parallel_calls=tf.data.AUTOTUNE)
test = test.batch(16)

## Посмотрим на содержимое набора данных

In [ ]:

images_and_masks = list(train.take(5))

fig, ax = plt.subplots(nrows = 2, ncols = 5, figsize=(15, 5), dpi=125)

for i, (image, masks) in enumerate(images_and_masks):
    ax[0, i].set_title('Image')
    ax[0, i].set_axis_off()
    ax[0, i].imshow(image)

    ax[1, i].set_title('Mask')
    ax[1, i].set_axis_off()
    ax[1, i].imshow(image)

    for channel in range(CLASSES):
        contours = measure.find_contours(np.array(masks[:, :, channel]))
        for contour in contours:
            ax[1, i].plot(contour[:, 1], contour[:, 0], linewidth=1, color=COLORS[channel])

plt.show()
plt.close()


## Обозначим основные блоки модели

In [ ]:
def input_layer():
    return tf.keras.layers.Input(shape=SAMPLE_SIZE + (3,))


def downsample_block(filters, size, batch_norm=True):
    model = tf.keras.Sequential()

    initializer = tf.keras.initializers.GlorotNormal()
    model.add(tf.keras.layers.Conv2D(filters, size, strides=2, padding='same', kernel_initializer=initializer, use_bias=False))

    if batch_norm:
        model.add(tf.keras.layers.BatchNormalization())

    model.add(tf.keras.layers.LeakyReLU())
    return model


def upsample_block(filters, size, dropout=False):
    model = tf.keras.Sequential()

    initializer = tf.keras.initializers.GlorotNormal()
    model.add(tf.keras.layers.Conv2DTranspose(filters, size, strides=2, padding='same', kernel_initializer=initializer, use_bias=False))

    model.add(tf.keras.layers.BatchNormalization())

    if dropout:
        model.add(tf.keras.layers.Dropout(0.25))

    model.add(tf.keras.layers.ReLU())
    return model


def output_layer(size):
    initializer = tf.keras.initializers.GlorotNormal()
    return tf.keras.layers.Conv2DTranspose(CLASSES, size, strides=2, padding='same', kernel_initializer=initializer, activation='sigmoid')

## Построим U-NET подобную архитектуру

In [ ]:
inp_layer = input_layer()

downsample_stack = [
    downsample_block(64, 4, batch_norm=False),
    downsample_block(128, 4),
    downsample_block(256, 4),
    downsample_block(512, 4),
    downsample_block(512, 4),
    downsample_block(512, 4),
    downsample_block(512, 4),
]

upsample_stack = [
    upsample_block(512, 4, dropout=True),
    upsample_block(512, 4, dropout=True),
    upsample_block(512, 4, dropout=True),
    upsample_block(256, 4),
    upsample_block(128, 4),
    upsample_block(64, 4)
]

out_layer = output_layer(4)


In [ ]:
# добавление skip-connections связей
x = inp_layer

downsample_skips = []

for block in downsample_stack:
    x = block(x)
    downsample_skips.append(x)

downsample_skips = reversed(downsample_skips[:-1])

for up_block, down_block in zip(upsample_stack, downsample_skips):
    x = up_block(x)
    x = tf.keras.layers.Concatenate()([x, down_block])

out_layer = out_layer(x)

unet_like = tf.keras.Model(inputs=inp_layer, outputs=out_layer)

tf.keras.utils.plot_model(unet_like, show_shapes=True, dpi=72)

## Определим метрики и функции потерь

In [ ]:
def dice_mc_metric(a, b):
    a = tf.unstack(a, axis=3)
    b = tf.unstack(b, axis=3)

    dice_summ = 0

    for i, (aa, bb) in enumerate(zip(a, b)):
        numenator = 2 * tf.math.reduce_sum(aa * bb) + 1
        denomerator = tf.math.reduce_sum(aa + bb) + 1
        dice_summ += numenator / denomerator

    avg_dice = dice_summ / CLASSES

    return avg_dice

def dice_mc_loss(a, b):
    return 1 - dice_mc_metric(a, b)

def dice_bce_mc_loss(a, b):
    return 0.3 * dice_mc_loss(a, b) + tf.keras.losses.binary_crossentropy(a, b)

## Компилируем модель

In [ ]:
unet_like.compile(optimizer='adam', loss=[tf.keras.losses.BinaryCrossentropy], metrics=['iou']) # tf.keras.losses.BinaryCrossentropy()

## Обучаем нейронную сеть и сохраняем результат

In [ ]:
history_dice = unet_like.fit(train, validation_data=valid, epochs=2, initial_epoch=0)

#unet_like.save_weights('SemanticSegmentationNetworks/unet_like')

## Загрузим модель

In [ ]:
unet_like.load_weights('SemanticSegmentationLesson/networks/unet_like')

## Проверим работу сети на всех кадрах из видео

In [ ]:
rgb_colors = [
    (0,   0,   0),
    (255, 0,   0),
    (0,   255, 0),
    (0,   0,   255),
    (255, 165, 0),
    (255, 192, 203),
    (0,   255, 255),
    (255, 0,   255)
]

frames = sorted(glob.glob('SemanticSegmentationLesson/videos/original_video/*.jpg'))

for filename in frames:
    frame = imread(filename)
    sample = resize(frame, SAMPLE_SIZE)

    predict = unet_like.predict(sample.reshape((1,) +  SAMPLE_SIZE + (3,)))
    predict = predict.reshape(SAMPLE_SIZE + (CLASSES,))

    scale = frame.shape[0] / SAMPLE_SIZE[0], frame.shape[1] / SAMPLE_SIZE[1]

    frame = (frame / 1.5).astype(np.uint8)

    for channel in range(1, CLASSES):
        contour_overlay = np.zeros((frame.shape[0], frame.shape[1]))
        contours = measure.find_contours(np.array(predict[:,:,channel]))

        try:
            for contour in contours:
                rr, cc = polygon_perimeter(contour[:, 0] * scale[0],
                                           contour[:, 1] * scale[1],
                                           shape=contour_overlay.shape)

                contour_overlay[rr, cc] = 1

            contour_overlay = dilation(contour_overlay, disk(1))
            frame[contour_overlay == 1] = rgb_colors[channel]
        except:
            pass

    imsave(f'SemanticSegmentationLesson/videos/processed/{os.path.basename(filename)}', frame)